<a href="https://colab.research.google.com/github/Eitan177/Bioinformatics_Bootcamp/blob/main/informatics_bootcamp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to the GOAL Bioinformatics Bootcamp, 2025! To Setup this notebook please run the first three cells. This will download the packages you will need to participate

In [1]:
# NGS Bioinformatics Hands On Lab

# Install required packages
!apt-get update
!apt-get install -y fastp fastqc bwa samtools tabix
!pip install pysam

# Optional: Install additional tools if not available in Colab
!wget https://github.com/biod/sambamba/releases/download/v0.8.0/sambamba-0.8.0-linux-amd64-static.gz
!gunzip sambamba-0.8.0-linux-amd64-static.gz
!chmod +x sambamba-0.8.0-linux-amd64-static
!mv sambamba-0.8.0-linux-amd64-static /usr/local/bin/sambamba

# For ABRA2
!wget https://github.com/mozack/abra2/releases/download/v2.23/abra2-2.23.jar
!chmod +x abra2-2.23.jar

# For VarScan
!wget https://github.com/dkoboldt/varscan/raw/master/VarScan.v2.4.6.jar
!chmod +x VarScan.v2.4.6.jar

# for ANNOVAR
!wget http://www.openbioinformatics.org/annovar/download/0wgxR2rIVP/annovar.latest.tar.gz
!tar -zxvf annovar.latest.tar.gz
!rm -r annovar.latest.tar.gz
!rm -r annovar/humandb
!perl annovar/annotate_variation.pl -buildver hg19 -downdb -webfrom annovar refGene assets/humandb/
!perl /content/annovar/annotate_variation.pl -buildver hg19 -downdb cytoBand assets/humandb/
!perl annovar/annotate_variation.pl -buildver hg19 -downdb -webfrom annovar clinvar_20150330 assets/humandb/
!perl annovar/annotate_variation.pl -buildver hg19 -downdb -webfrom annovar cosmic85 assets/humandb/

# Create directories for data and reference files
!mkdir -p data assets/gatk_b37 assets/humandb annovar

# Download pre-indexed reference genome
!wget https://hgdownload.cse.ucsc.edu/goldenpath/hg19/bigZips/analysisSet/hg19.p13.plusMT.no_alt_analysis_set.bwa_index.tar.gz -O assets/hg19_bwa_index.tar.gz



Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,607 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,705 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jamm

In [2]:
!wget https://hgdownload.cse.ucsc.edu/goldenpath/hg19/bigZips/analysisSet/hg19.p13.plusMT.no_alt_analysis_set.fa.gz -O /content/assets/gatk_b37/hg19.p13.plusMT.no_alt_analysis_set.fa.gz

--2025-05-01 12:55:26--  https://hgdownload.cse.ucsc.edu/goldenpath/hg19/bigZips/analysisSet/hg19.p13.plusMT.no_alt_analysis_set.fa.gz
Resolving hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)|128.114.119.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 863259336 (823M) [application/x-gzip]
Saving to: ‘/content/assets/gatk_b37/hg19.p13.plusMT.no_alt_analysis_set.fa.gz’

/content/assets/gat 100%[===================>] 823.27M  83.0MB/s    in 7.4s    

2025-05-01 12:55:33 (112 MB/s) - ‘/content/assets/gatk_b37/hg19.p13.plusMT.no_alt_analysis_set.fa.gz’ saved [863259336/863259336]



In [3]:
# Extract BWA index files
!tar -xzvf /content/assets/hg19_bwa_index.tar.gz -C /content/assets/gatk_b37/



hg19.p13.plusMT.no_alt_analysis_set/hg19.p13.plusMT.no_alt_analysis_set.fa.amb
hg19.p13.plusMT.no_alt_analysis_set/hg19.p13.plusMT.no_alt_analysis_set.fa.ann
hg19.p13.plusMT.no_alt_analysis_set/hg19.p13.plusMT.no_alt_analysis_set.fa.bwt
hg19.p13.plusMT.no_alt_analysis_set/hg19.p13.plusMT.no_alt_analysis_set.fa.pac
hg19.p13.plusMT.no_alt_analysis_set/hg19.p13.plusMT.no_alt_analysis_set.fa.sa


In [4]:
!cp /content/assets/gatk_b37/hg19.p13.plusMT.no_alt_analysis_set/* /content/assets/gatk_b37/

In [5]:
# Uncompress the reference FASTA file
!gunzip -c /content/assets/gatk_b37/hg19.p13.plusMT.no_alt_analysis_set.fa.gz > /content/assets/gatk_b37/hg19.p13.plusMT.no_alt_analysis_set.fa


In [6]:
!wget https://raw.githubusercontent.com/Eitan177/Bioinformatics_Bootcamp/main/ngs_ws_hd701_R1.fq.gz
!wget https://raw.githubusercontent.com/Eitan177/Bioinformatics_Bootcamp/main/ngs_ws_hd701_R2.fq.gz
!wget https://raw.githubusercontent.com/Eitan177/Bioinformatics_Bootcamp/main/ngs_ws.bed
!cp ngs_ws.bed /content/assets/

--2025-05-01 12:58:17--  https://raw.githubusercontent.com/Eitan177/Bioinformatics_Bootcamp/main/ngs_ws_hd701_R1.fq.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 153947 (150K) [application/octet-stream]
Saving to: ‘ngs_ws_hd701_R1.fq.gz’

ngs_ws_hd701_R1.fq. 100%[===================>] 150.34K  --.-KB/s    in 0.04s   

2025-05-01 12:58:18 (3.86 MB/s) - ‘ngs_ws_hd701_R1.fq.gz’ saved [153947/153947]

--2025-05-01 12:58:18--  https://raw.githubusercontent.com/Eitan177/Bioinformatics_Bootcamp/main/ngs_ws_hd701_R2.fq.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent,

In [7]:
!wget https://raw.githubusercontent.com/Eitan177/Bioinformatics_Bootcamp/main/ngs_ws.bed
!cp ngs_ws.bed /content/assets/

--2025-05-01 13:02:03--  https://raw.githubusercontent.com/Eitan177/Bioinformatics_Bootcamp/main/ngs_ws.bed
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 204 [text/plain]
Saving to: ‘ngs_ws.bed.1’

ngs_ws.bed.1        100%[===================>]     204  --.-KB/s    in 0s      

2025-05-01 13:02:03 (10.9 MB/s) - ‘ngs_ws.bed.1’ saved [204/204]



In [8]:
# 1. Examine content of the FASTQ file
!zcat /content/ngs_ws_hd701_R1.fq.gz | head -n 20

# 2. Perform FASTQ processing prior to alignment
!fastp -i /content/ngs_ws_hd701_R1.fq.gz -o data/hd701_R1_processed.fq.gz -I /content/ngs_ws_hd701_R2.fq.gz -O data/hd701_R2_processed.fq.gz -z 4 -w 2 -h data/hd701_fq_qc.html

# 3. Generate FASTQ quality control data
!fastqc -o data -f fastq /content/ngs_ws_hd701_R1.fq.gz /content/ngs_ws_hd701_R2.fq.gz

# 4. Display QC report in Colab
from IPython.display import HTML
with open('data/hd701_fq_qc.html', 'r') as f:
    html_content = f.read()
HTML(html_content)


@NB501446:28:HMG5HBGX2:1:11101:10612:17818/1
GTCAATAATTCTGTTCTTAGGCCACTTTTCTCCCTTAATTTATTTCCTGTTCTTTCTCTTTTTAGCTCCCAGGGTT
+
AAAAAEEEEEEEEEEEEEEEEEEEEEEEEEEEEAAEEEEEEEEEEEAEEEEEEEEEEEEEEEEEEEEEE6EEEEEE
@NB501446:28:HMG5HBGX2:1:11101:16871:6164/1
GCCATTGCAGTGCAGAAGGGCAGTATCGGCAACCTCCTGGGCTTCCTAGAGGACAGCCTTGCCGCCACACAGTTCC
+
AAAAAEEEEEEEEEEAAEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE/EE/E<EEEEEEEEEEE/EEAAAEAEE
@NB501446:28:HMG5HBGX2:1:11101:17659:14137/1
GTTTGCTGGCCTGTTGAAAAATGACTGTAACAAAAGTGCTTCTGGTTATTTAACAGATGAAAATGAAGTGGGGTTT
+
AAAAAEEEEEEEEEEEEEAEEEEEEEEEEEEEEEEEEEEEEEEEEEEAEEEEEEEEEEEEEEEEEEEEEEEEEEEE
@NB501446:28:HMG5HBGX2:1:11101:23347:13640/1
ATATGTAAATGTAGTATAGGGAAGCTTCATAAGTCAGTCTCATCTGCAAATACTTGTGGGATTTTTAGCACAGCAA
+
AAAAAEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE
@NB501446:28:HMG5HBGX2:1:11101:7116:18158/1
CCGCCTGGAGAAGTTGTATAGCACTATGGTGCGCTTCCTCAGTGACCGAAAGAACCCGGTGTGCCGGGAGATGGCT
+
AAAAAAEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEAEEE

fastp version:,0.20.1 (https://github.com/OpenGene/fastp)
sequencing:,paired end (76 cycles + 76 cycles)
mean length before filtering:,"75bp, 76bp"
mean length after filtering:,"75bp, 75bp"
duplication rate:,7.811206%
Insert size peak:,117
total reads:,10.052000 K
total bases:,763.919000 K
Q20 bases:,743.785000 K (97.364380%)
Q30 bases:,734.827000 K (96.191743%)
GC content:,41.940441%


In [9]:
!bwa mem -M -v 1 -t 2 -R "@RG\\tID:hd701\\tSM:hd701\\tPL:ILLUMINA\\tPI:150\\tCN:lab" /content/assets/gatk_b37/hg19.p13.plusMT.no_alt_analysis_set.fa /content/data/hd701_R1_processed.fq.gz /content/data/hd701_R2_processed.fq.gz > /content/data/hd701_raw.sam

[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (144, 177, 224)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (1, 384)
[M::mem_pestat] mean and std.dev: (186.75, 58.21)
[M::mem_pestat] low and high boundaries for proper pairs: (1, 464)
[M::mem_pestat] skip orientation RF as there are not enough pairs
[M::mem_pestat] skip orientation RR as there are not enough pairs
[main] Version: 0.7.17-r1188
[main] CMD: bwa mem -M -v 1 -t 2 -R @RG\tID:hd701\tSM:hd701\tPL:ILLUMINA\tPI:150\tCN:lab /content/assets/gatk_b37/hg19.p13.plusMT.no_alt_analysis_set.fa /content/data/hd701_R1_processed.fq.gz /content/data/hd701_R2_processed.fq.gz
[main] Real time: 24.047 sec; CPU: 10.097 sec


In [10]:
# 6. Convert the SAM file into a BAM file
!samtools view -Shu data/hd701_raw.sam > data/hd701_raw.bam

# 7. Sort and index raw BAM file
!sambamba sort -p -t 2 -o data/hd701_sorted.bam data/hd701_raw.bam

# 8. Mark PCR duplicates in BAM file
!sambamba markdup -t 2 -p data/hd701_sorted.bam data/hd701_dedup.bam


sambamba 0.8.0
 by Artem Tarasov and Pjotr Prins (C) 2012-2020
    LDC 1.10.0 / DMD v2.080.1 / LLVM6.0.1 / bootstrap LDC - the LLVM D compiler (0.17.4)

Writing sorted chunks to temporary directory...
[==============================================================================]

sambamba 0.8.0
 by Artem Tarasov and Pjotr Prins (C) 2012-2020
    LDC 1.10.0 / DMD v2.080.1 / LLVM6.0.1 / bootstrap LDC - the LLVM D compiler (0.17.4)

finding positions of the duplicate reads in the file...
[==============================================================================]
  sorted 4959 end pairs
     and 0 single ends (among them 0 unmatched pairs)
  collecting indices of duplicate reads...   done in 5 ms
  found 860 duplicates
collected list of positions in 0 min 0 sec
marking duplicates...
[==============================================================================]
collected list of positions in 0 min 0 sec


In [11]:
# 9. Perform indel realignment - Update the reference path
!java -jar abra2-2.23.jar --in data/hd701_dedup.bam --out data/hd701_realigned.bam --ref /content/assets/gatk_b37/hg19.p13.plusMT.no_alt_analysis_set.fa \\
!--threads 2 --targets assets/ngs_ws.bed --index --tmpdir /tmp/ > data/hd701_abra.log

# 10. Inspect aligned sequences (show first few lines)
!samtools view -h data/hd701_realigned.bam | head -n 20



INFO	Thu May 01 13:16:31 UTC 2025	Abra version: 2.23
INFO	Thu May 01 13:16:31 UTC 2025	Abra params: [/content/abra2-2.23.jar --in data/hd701_dedup.bam --out data/hd701_realigned.bam --ref /content/assets/gatk_b37/hg19.p13.plusMT.no_alt_analysis_set.fa  !--threads 2 --targets assets/ngs_ws.bed --index --tmpdir /tmp/]
INFO	Thu May 01 13:16:31 UTC 2025	ABRA version: 2.23
INFO	Thu May 01 13:16:31 UTC 2025	input0: data/hd701_dedup.bam
INFO	Thu May 01 13:16:31 UTC 2025	output0: data/hd701_realigned.bam
INFO	Thu May 01 13:16:31 UTC 2025	regions: assets/ngs_ws.bed
INFO	Thu May 01 13:16:31 UTC 2025	reference: /content/assets/gatk_b37/hg19.p13.plusMT.no_alt_analysis_set.fa
INFO	Thu May 01 13:16:31 UTC 2025	num threads: 4
INFO	Thu May 01 13:16:31 UTC 2025	minEdgeFrequency: 0
minNodeFrequncy: 1
minContigLength: -1
minBaseQuality: 20
minReadCandidateFraction: 0.01
maxAverageRegionDepth: 1000
minEdgeRatio: 0.01

INFO	Thu May 01 13:16:31 UTC 2025	paired end: true
INFO	Thu May 01 13:16:31 UTC 2025	isS

In [12]:
!pip install -U gdown
import gdown
file_id = "1yANVV31SMqwzRNnEFiLSktxj8eQL5CSp"
gdown.download(f"https://drive.google.com/uc?id={file_id}", quiet=False)
#!wget https://drive.google.com/file/d/1yANVV31SMqwzRNnEFiLSktxj8eQL5CSp/view?usp=sharing

Downloading...
From (original): https://drive.google.com/uc?id=1yANVV31SMqwzRNnEFiLSktxj8eQL5CSp
From (redirected): https://drive.google.com/uc?id=1yANVV31SMqwzRNnEFiLSktxj8eQL5CSp&confirm=t&uuid=7545913b-7693-4a76-8c85-c61984c4e49b
To: /content/annotation_db_and_vcfs.zip
100%|██████████| 238M/238M [00:04<00:00, 52.5MB/s]


'annotation_db_and_vcfs.zip'

In [13]:
!unzip annotation_db_and_vcfs.zip

Archive:  annotation_db_and_vcfs.zip
  inflating: grip_course_annovar_db.tar.gz  
  inflating: grip_course_annovar_db.tar.gz.md5  
  inflating: grip_vcfs.tar.gz        
  inflating: grip_vcfs.tar.gz.md5    


In [14]:
!tar -xzvf /content/grip_course_annovar_db.tar.gz

grip_course_annovar_db/
grip_course_annovar_db/hg19_wgRna.txt
grip_course_annovar_db/hg19_refGeneVersion.txt
grip_course_annovar_db/hg19_cosmic85.txt.idx
grip_course_annovar_db/hg19_clinvar_20150330.txt
grip_course_annovar_db/hg19_exac03nontcga.txt.idx
grip_course_annovar_db/hg19_refGeneMrna.fa
grip_course_annovar_db/hg19_exac03nontcga.txt
grip_course_annovar_db/hg19_refGene.txt
grip_course_annovar_db/hg19_cytoBand.txt
grip_course_annovar_db/hg19_cosmic85.txt
grip_course_annovar_db/hg19_clinvar_20150330.txt.idx


In [16]:
!cp /content/grip_course_annovar_db/* /content/assets/humandb/

In [17]:
!samtools index data/hd701_realigned.bam
!ls -l /content/assets/gatk_b37/hg19.p13.plusMT.no_alt_analysis_set.fa
!head -n 5 /content/assets/gatk_b37/hg19.p13.plusMT.no_alt_analysis_set/hg19.fa
!ls -l assets/ngs_ws.bed
!head -n 5 assets/ngs_ws.bed
!java -version
!ls -l VarScan.v2.4.6.jar


-rw-r--r-- 1 root root 3146301083 May  1 12:58 /content/assets/gatk_b37/hg19.p13.plusMT.no_alt_analysis_set.fa
head: cannot open '/content/assets/gatk_b37/hg19.p13.plusMT.no_alt_analysis_set/hg19.fa' for reading: No such file or directory
-rw-r--r-- 1 root root 204 May  1 13:02 assets/ngs_ws.bed
chr1	27105337	27107952	ARID1A
chr3	178951769	178952287	PIK3CA_ex21
chr7	55241584	55241766	EGFR_ex18
chr7	140453045	140453223	BRAF_ex15
chr12	25398208	25398329	KRAS_ex2
openjdk version "11.0.26" 2025-01-21
OpenJDK Runtime Environment (build 11.0.26+4-post-Ubuntu-1ubuntu122.04)
OpenJDK 64-Bit Server VM (build 11.0.26+4-post-Ubuntu-1ubuntu122.04, mixed mode, sharing)
-rwxr-xr-x 1 root root 126611 May  1 12:54 VarScan.v2.4.6.jar


In [18]:
!samtools mpileup -BA -q 20 -Q 30 -d 4000 -l assets/ngs_ws.bed -f /content/assets/gatk_b37/hg19.p13.plusMT.no_alt_analysis_set.fa data/hd701_realigned.bam > data/hd701_mpileup.txt
!wc -l data/hd701_mpileup.txt
# Step 2: Run VarScan on the mpileup file
#!cat data/hd701_mpileup.txt | java -Xmx4G -jar VarScan.v2.4.6.jar mpileup2vcf --min-coverage 8 --min-var-freq 0.05 --p-value 0.05 --min-avg-qual 30 --strand-filter 1 --output-vcf 1 --variants > data/hd701_raw.vcf

[mpileup] 1 samples in 1 input files
8078 data/hd701_mpileup.txt


In [19]:
!# 11. Call variant - Update the reference path
!samtools mpileup -BA -q 20 -Q 30 -d 4000 -l assets/ngs_ws.bed -f /content/assets/gatk_b37/hg19.p13.plusMT.no_alt_analysis_set.fa data/hd701_realigned.bam | java -Xmx4G -jar VarScan.v2.4.6.jar mpileup2vcf --min-coverage 8 --min-var-freq 0.05 --p-value 0.05 --min-avg-qual 30 --strand-filter 1 --output-vcf 1 --variants > data/hd701_raw.vcf

[mpileup] 1 samples in 1 input files
Only variants will be reported
Min coverage:	8
Min reads2:	2
Min var freq:	0.05
Min avg qual:	30
P-value thresh:	0.05
Input stream not ready, waiting for 5 seconds...
Reading input from STDIN
8078 bases in pileup file
14 variant positions (10 SNP, 4 indel)
0 were failed by the strand-filter
14 variant positions reported (10 SNP, 4 indel)


In [20]:
# 12. Compress and index VCF file
!bgzip data/hd701_raw.vcf
!tabix -p vcf data/hd701_raw.vcf.gz

# 13. Inspect variants in a raw VCF file
!zcat data/hd701_raw.vcf.gz | head -n 20



##fileformat=VCFv4.3
##source=VarScan2
##INFO=<ID=ADP,Number=1,Type=Integer,Description="Average per-sample depth of bases with Phred score >= 30">
##INFO=<ID=WT,Number=1,Type=Integer,Description="Number of samples called reference (wild-type)">
##INFO=<ID=HET,Number=1,Type=Integer,Description="Number of samples called heterozygous-variant">
##INFO=<ID=HOM,Number=1,Type=Integer,Description="Number of samples called homozygous-variant">
##INFO=<ID=NC,Number=1,Type=Integer,Description="Number of samples not called">
##FILTER=<ID=str10,Description="Less than 10% or more than 90% of variant supporting reads on one strand">
##FILTER=<ID=indelError,Description="Likely artifact due to indel reads at this position">
##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">
##FORMAT=<ID=GQ,Number=1,Type=Integer,Description="Genotype Quality">
##FORMAT=<ID=SDP,Number=1,Type=Integer,Description="Raw Read Depth as reported by SAMtools">
##FORMAT=<ID=DP,Number=1,Type=Integer,Description="Quality

In [21]:
!perl /content/annovar/annotate_variation.pl -buildver hg19 -downdb hg19_cosmic assets/humandb/

NOTICE: Web-based checking to see whether ANNOVAR new version is available ... Done
NOTICE: Downloading annotation database http://hgdownload.cse.ucsc.edu/goldenPath/hg19/database/hg19_cosmic.txt.gz ... Failed


In [ ]:
# 14. Annotate variants (requires ANNOVAR to be set up)
!perl annovar/table_annovar.pl data/hd701_raw.vcf.gz assets/humandb/ -out data/hd701_annotated -buildver hg19 -remove -nastring . -otherinfo -vcfinput -thread 2 -maxgenethread 2 -protocol refGene,cytoBand,cosmic85,clinvar_20150330 -operation g,r,f,f

# 15. Inspect annotated VCF file
!head -n 20 data/hd701_annotated.hg19_multianno.vcf

NOTICE: the --polish argument is set ON automatically (use --nopolish to change this behavior)
Error: the required database file assets/humandb/hg19_cosmic85.txt does not exist.
head: cannot open 'data/hd701_annotated.hg19_multianno.vcf' for reading: No such file or directory
